In [1]:
using GradientRobustMultiPhysics
using ExtendableGrids
using GridVisualize
using ExtendableSparse
using Printf
include("../src/flowdata.jl")

get_flow_data (generic function with 1 method)

## Transien Stokes
The idea is to implement the adaptive time step for the transient Stokes case. The weak and discrete form should be detailed later

In [41]:
function run_main(; scheme=1, ν=1, nlev=1, nsteps=10)
  xgrid = get_grid(2)
  #get_problemdata(problem; Re = 1, nonlinear = true, )
  u, p, f, u_t= get_flow_data(ν)
  ## choose one of these (inf-sup stable) finite element type pairs
  FETypes = [H1P2{2,2}, H1P1{1}] # Taylor-Hood elements
  # generate FE spaces
  FES = [FESpace{FETypes[1]}(xgrid), FESpace{FETypes[2]}(xgrid)]
  # solution vector
  Solution = FEVector(["u_h", "p_h"], FES)
  # dofs 
  ndofu = FES[1].ndofs
  ndofp = FES[2].ndofs
  # initial solution
  GradientRobustMultiPhysics.interpolate!(Solution[1], u; time = 0.)

  # mass matrix 
  M = FEMatrix{Float64}(FES)
  assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
  # velocity pressure matrices
  A = FEMatrix{Float64}(FES)
  assemble_operator!(A[1,1], LaplaceOperator(ν))
  assemble_operator!(A[1,2], LagrangeMultiplier(Divergence); At = A[2,1]) 

  rhs = FEVector{Float64}(FES)
  assemble_operator!(rhs[1], LinearForm(Identity, f); time=0.0)
  t0 = zero(Float64)
  Tf = 1.0
  tau = Tf/nsteps
  nts=1
  #------------------------------ time loop ----------------------------
  SystemMatrix = FEMatrix(FES)
  # @show SystemMatrix
  SystemRHS = FEVector(FES)
  SystemSol = FEVector(FES)
  V1 = zeros(Float64, ndofu, 1)
  while t0 < Tf-1e-10
    t0 = t0 + tau
    fill!(SystemRHS.entries, 0)
    fill!(rhs.entries, 0)
    assemble_operator!(rhs[1], LinearForm(Identity, f), time= t0 )
    V1[:, 1] = rhs[1][:]
    fill!(SystemMatrix.entries.cscmatrix.nzval, 0)
    fill!(A.entries.cscmatrix.nzval, 0)
    if nts == 1
      if scheme == 2
        @info("First step with BDF1")
      end
      fill!(A[1,1], 0)
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time= t0 )
      addblock!(SystemMatrix[1, 1], M[1, 1]; factor= 1.0/tau)
      #addblock!(SystemMatrix[2, 2], M[1, 1]; factor= 1.0/tau)
      addblock!(SystemMatrix[1,1], A[1, 1]; factor= 1.0)
      # addblock!(SystemMatrix[2,2], A[1, 1]; factor= 1.0)
      addblock!(SystemMatrix[1,2], A[1, 2]; factor= 1.0)
      addblock!(SystemMatrix[2,1], A[2, 1]; factor= 1.0)
    end
    l2 = 0.0
    @printf("it:%d: [%.5f, %.5e, %5.5e]\n", nts, t0, l2, l2)
  end
end

run_main (generic function with 1 method)

In [42]:
run_main()

it:1: [0.10000, 0.00000e+00, 0.00000e+00]
it:1: [0.20000, 0.00000e+00, 0.00000e+00]
it:1: [0.30000, 0.00000e+00, 0.00000e+00]
it:1: [0.40000, 0.00000e+00, 0.00000e+00]
it:1: [0.50000, 0.00000e+00, 0.00000e+00]
it:1: [0.60000, 0.00000e+00, 0.00000e+00]
it:1: [0.70000, 0.00000e+00, 0.00000e+00]
it:1: [0.80000, 0.00000e+00, 0.00000e+00]
it:1: [0.90000, 0.00000e+00, 0.00000e+00]
it:1: [1.00000, 0.00000e+00, 0.00000e+00]


┌ Info: Setting up uniform grid 
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/src/flowdata.jl:11
